# Pizza maker

Specify the topping to create the texture for your favorite pizza.

In [ ]:
import os
assets_dir = os.path.join(os.path.abspath("."), 'assets') 

## How to use
Edit `YOUR_PIZZA_NAME`, `BASE_PIZZA`, and the `TOPPING` list in the cell below, then run it to generate the pizza. The distribution of toppings is randomic, re-run the cell to get a new distribution. When satisfied, run the last cell to save the image.

In [ ]:
import math
import random
from PIL import Image

### USER INPUT
# Create your pizza here!
YOUR_PIZZA_NAME = "quattro stagioni"
BASE_PIZZA = "margherita"               # must be in `assets/pizzas`
TOPPINGS = ["mushrooms", "ham", "artichokes", "olives"]    # must be in `assets/toppings`
SIZE = (1024, 1024) # size of the final image
SOURCE = 'default' # must be `default` or `custom`
###

if not SOURCE == 'custom':
        # Default images are prepended with '_'.
        prefix = '_'
#random.seed(1) # for reproducibility, change this to change your pizza
                # comment to have a new pizza at each run

# Get the available toppings
toppings_available = []
for f in os.listdir(os.path.join(assets_dir, "toppings")):
    # To be "available" a topping must have an associated PNG image in the `assets/toppings` folder
    if f.endswith(".png"):
        topping = f.split(".")[0].lstrip("_")
        toppings_available.append(topping)

# Import the 'base' pizza image
bg_img = Image.open(os.path.join(assets_dir,
                                 "pizzas",
                                 "{}{}.png".format(prefix, BASE_PIZZA))
                              )
bg_img = bg_img.resize(SIZE)  # size of the original "margherita.png"

bg_w, bg_h = bg_img.size
D = min(bg_w, bg_h)

# Scatter topping images onto the pizza image. The toppings are placed into concencric circles. A small random offset is added to each topping.
R = D//2 * 0.55     # adjust the gains to have toppings 
r = D//2 * 0.25     # closer or further from center.
scale = 1.0 # scale factor for topping images
max_scattering = D//2 * 0.15  # [px] maximum scattering wrt to seed position. 
for topping in TOPPINGS:
    if topping not in toppings_available:
        print("'{}' is not in the available toppings.\n".format(topping.capitalize()) + 
        "The topping will not be added to your pizza.\n" +
        "Please add the corresponding `{}{}.png` image in the `assets` dir to make it available.".format(prefix, topping))
        continue
    top_img = Image.open(os.path.join(assets_dir,
                                      "toppings",
                                      "{}{}.png".format(prefix, topping)))
    top_w, top_h = top_img.size
    d = int(top_w * scale)
    top_img = top_img.resize((d, d))
    for i in range(-4, 4):
        ang = math.pi * i / 4
        # Each offset coordinate is made of three terms:
        #   - nominal position (add toppings regularly, in concentric circles)
        #   - random factor ("scatters" topping from nominal position)
        #   - icon size / 2 (move nominal position to topping center rather than corner)
        offset = (bg_w //2 + int(math.sin(ang) * R) +
                  int((random.random()-0.5)*max_scattering) -
                  d // 2, 
                  bg_h // 2 + int(math.cos(ang) * R) + 
                  int((random.random()-0.5)*max_scattering) -
                  d // 2,
                  )
        bg_img.paste(top_img, offset, top_img)
        offset = (bg_w // 2 + int(math.sin(ang) * r) + 
                  int((random.random()-0.5)*max_scattering) -
                  d // 2, 
                  bg_h // 2 + int(math.cos(ang) * r) + 
                  int((random.random()-0.5)*max_scattering) -
                  d // 2,
                  )
        bg_img.paste(top_img, offset, top_img) 
# Display the result
bg_img
# If you are satisfied, move to the next cell to save your pizza.
# Save-by-default is avoided to prevent overwriting.

In [ ]:
# Save your pizza
bg_img.save(os.path.join(assets_dir,
                         "pizzas",
                         "{}{}.png".format(prefix,
                                           YOUR_PIZZA_NAME.lower().replace(" ",
                                                                           "_")
            )))